In [52]:
import pandas as pd
import requests
import tweepy
from datetime import datetime
#NLP libraries 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from textblob import TextBlob
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/owenkinsella/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/owenkinsella/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
#Check errors in date column formatting
def validate(date_text):
    errors = []
    for i,v in enumerate(date_text):
        try:
            datetime.strptime(v, '%Y-%m-%d %H:%M:%S')
        except ValueError:
            errors.append(i)
    return errors

In [50]:
def clean_df(df,hashtags):
    #drop null values and duplicates
    df.dropna(axis=0 ,subset=['date','text'],inplace = True)
    df.drop_duplicates(inplace = True)
    df.reset_index(drop= True,inplace=True)
    #convert to most relevant dtypes
    df = df.convert_dtypes()
    #check number of dates with inconisitent format
    errors = validate(df['date'])
    #remove dates with inconisitent format
    df_clean = df.loc[~df.index.isin(errors)]
    df_clean.reset_index(drop=True,inplace=True)
    #convert dtype to datetime for dates and shorten to YYYY-MM-DD
    df_clean['date'] = pd.to_datetime(df_clean.loc[:,'date'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d')
    #remove irrelevant hashtags
    df_clean['hashtags'] = df_clean['hashtags'].str.lower()
    df_clean = df_clean[df_clean['hashtags'].str.contains('|'.join(hashtags)) == False].reset_index(drop = True)
    #remove unicode characters
    df_clean['text'] = df_clean['text'].str.replace("(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"," ")
    #tokenise text and remove stopwords
    df_clean['text'] = df_clean.apply(lambda row: word_tokenize(row['text']), axis=1)
    stop_words = set(stopwords.words('english'))
    df_clean['text'] = df_clean.apply(lambda row: (" ".join([word for word in row['text'] if word not in stop_words])),axis=1)
    return df_clean

In [36]:
chunk = pd.read_csv('Bitcoin_tweets.csv',chunksize=100000,lineterminator='\n')
df_sentiment2 = pd.read_csv('Bitcoin_tweets_dataset_2.csv',lineterminator='\n')

/var/folders/zk/_8g_vn511sq_ckc0yy4ngm8m0000gn/T/ipykernel_34476/1353951192.py:2: DtypeWarning: Columns (5,6,7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sentiment1 = pd.concat(chunk)
/var/folders/zk/_8g_vn511sq_ckc0yy4ngm8m0000gn/T/ipykernel_34476/1353951192.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sentiment1 = pd.concat(chunk)
/var/folders/zk/_8g_vn511sq_ckc0yy4ngm8m0000gn/T/ipykernel_34476/1353951192.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sentiment1 = pd.concat(chunk)
/var/folders/zk/_8g_vn511sq_ckc0yy4ngm8m0000gn/T/ipykernel_34476/1353951192.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sentiment1 = pd.concat(chunk)
/var/folders/zk/_8g_vn511sq_ckc0yy4ngm8m0000gn/T/ipykernel_34476/1353951192.py:2: DtypeWarning: Columns (5,6,7) have mixed

In [54]:
hashtags = ['nft','giveaway','cashback','airdrop']
df_clean = clean_df(df_sentiment1,hashtags)

/var/folders/zk/_8g_vn511sq_ckc0yy4ngm8m0000gn/T/ipykernel_34476/178301672.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['date'] = pd.to_datetime(df_clean.loc[:,'date'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d')
/var/folders/zk/_8g_vn511sq_ckc0yy4ngm8m0000gn/T/ipykernel_34476/178301672.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['hashtags'] = df_clean['hashtags'].str.lower()
/var/folders/zk/_8g_vn511sq_ckc0yy4ngm8m0000gn/T/ipykernel_34476/178301672.py:19: FutureWa

In [55]:
df_clean.to_csv('Bitcoin_tweets_clean.csv',chunksize=100000,lineterminator='\n')

In [56]:
pd.date_range(start=df_clean['date'].min(),end=df_clean['date'].max(),freq='D')

DatetimeIndex(['2021-02-05', '2021-02-06', '2021-02-07', '2021-02-08',
               '2021-02-09', '2021-02-10', '2021-02-11', '2021-02-12',
               '2021-02-13', '2021-02-14',
               ...
               '2022-12-31', '2023-01-01', '2023-01-02', '2023-01-03',
               '2023-01-04', '2023-01-05', '2023-01-06', '2023-01-07',
               '2023-01-08', '2023-01-09'],
              dtype='datetime64[ns]', length=704, freq='D')